# CSE 4510 MongoDB Extra Credit
## Topic: MongoDB NoSQL
**Name: Autumn Monsees** | Due Date: 11/26/2022

### Preamble
Import Packages

In [1]:
import pandas as pd
import pymongo
import json

### Part 1 - Data Cleaning, Transformation, and Storage

**a) Use the code and instructions provided by the instructor during the in-class demo to convert the dataset to a proper DataFrame having the following columns:**
- DATE, TIME_UTC, POINT_TYPE, STATUS, LATITUDE, LONGITUDE, MAX_WINDSPEED_KT, MIN_PRESURE_MB, NE_34KT, SE_34KT, NW_34_KT, SW_34_KT, NE_50KT, SE_50KT, NW_50_KT, SW_50_KT, NE_64KT, SE_64KT, NW_64_KT, SW_64_KT, RADIUS, BASIN, ATCF_CYCLONE_NUMBER, YEAR, NAME, NUM_BEST_TRACK_ENTRIES

Get the columns in a list

In [2]:
columns_str = "DATE, TIME_UTC, POINT_TYPE, STATUS, LATITUDE, LONGITUDE, MAX_WINDSPEED_KT, MIN_PRESSURE_MB, NE_34KT, SE_34KT, NW_34_KT, SW_34_KT, NE_50KT, SE_50KT, NW_50_KT, SW_50_KT, NE_64KT, SE_64KT, NW_64_KT, SW_64_KT, RADIUS, BASIN, ATCF_CYCLONE_NUMBER, YEAR, NAME, NUM_BEST_TRACK_ENTRIES"

In [3]:
columns = columns_str.replace(' ', '').split(",")

In [4]:
columns

['DATE',
 'TIME_UTC',
 'POINT_TYPE',
 'STATUS',
 'LATITUDE',
 'LONGITUDE',
 'MAX_WINDSPEED_KT',
 'MIN_PRESSURE_MB',
 'NE_34KT',
 'SE_34KT',
 'NW_34_KT',
 'SW_34_KT',
 'NE_50KT',
 'SE_50KT',
 'NW_50_KT',
 'SW_50_KT',
 'NE_64KT',
 'SE_64KT',
 'NW_64_KT',
 'SW_64_KT',
 'RADIUS',
 'BASIN',
 'ATCF_CYCLONE_NUMBER',
 'YEAR',
 'NAME',
 'NUM_BEST_TRACK_ENTRIES']

In [5]:
storms = dict({})
header = None
with open("hurdat2-1851-2021-100522.txt", 'r') as f:
    for l in f:
        split_l = l.split(',')
        if len(split_l) == 4:
            header = l
            storms[l] = []
        else:
            storms[header].append([d.strip() for d in split_l])
            date = storms[header][-1][0]
            storms[header][-1][0] = f"{date[:4]}-{date[4:6]}-{date[-2:]}"

In [6]:
frames = []
for storm in storms:
    id_str, name, number, _ = [s.strip() for s in storm.split(',')]
    basin = id_str[:2]
    id_no = id_str[2:-4]
    year  = id_str[-4:]
    
    storm_dict = {"BASIN": basin, "ATCF_CYCLONE_NUMBER": id_no, "YEAR": year, "NAME":name, "NUM_BEST_TRACK_ENTRIES": number}
    storm_data = pd.DataFrame(storms[storm], columns=columns[:-5])
    for i in storm_dict:
        storm_data[i] = storm_dict[i]
    frames.append(storm_data)

In [7]:
final_df = pd.concat(frames)

In [8]:
final_df

,DATE,TIME_UTC,POINT_TYPE,STATUS,LATITUDE,LONGITUDE,MAX_WINDSPEED_KT,MIN_PRESSURE_MB,NE_34KT,SE_34KT,...,NE_64KT,SE_64KT,NW_64_KT,SW_64_KT,RADIUS,BASIN,ATCF_CYCLONE_NUMBER,YEAR,NAME,NUM_BEST_TRACK_ENTRIES
0,1851-06-25,0000,,HU,28.0N,94.8W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
1,1851-06-25,0600,,HU,28.0N,95.4W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
2,1851-06-25,1200,,HU,28.0N,96.0W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
3,1851-06-25,1800,,HU,28.1N,96.5W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
4,1851-06-25,2100,L,HU,28.2N,96.8W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,2021-11-07,0000,,TS,37.4N,37.4W,35,1003,0,60,...,0,0,0,0,40,AL,21,2021,WANDA,54
50,2021-11-07,0600,,TS,38.1N,36.4W,35,1004,0,60,...,0,0,0,0,45,AL,21,2021,WANDA,54
51,2021-11-07,1200,,LO,39.2N,34.9W,35,1006,0,90,...,0,0,0,0,50,AL,21,2021,WANDA,54
52,2021-11-07,1800,,LO,40.9N,32.8W,40,1006,0,90,...,0,0,0,0,50,AL,21,2021,WANDA,54


**b) Convert the latitudes and longitudes from hemispheric values (NSEW) to float values (southern and
western values should be negative). Display the head of your dataset to show the changes.**

In [9]:
def get_lat_long(entry):
    return float(entry[:-1]) * (-1 if entry[-1] in "SW" else 1)

In [10]:
final_df['LATITUDE'] = final_df['LATITUDE'].apply(get_lat_long)

In [11]:
final_df['LONGITUDE'] = final_df['LONGITUDE'].apply(get_lat_long)

In [12]:
final_df.head()

,DATE,TIME_UTC,POINT_TYPE,STATUS,LATITUDE,LONGITUDE,MAX_WINDSPEED_KT,MIN_PRESSURE_MB,NE_34KT,SE_34KT,...,NE_64KT,SE_64KT,NW_64_KT,SW_64_KT,RADIUS,BASIN,ATCF_CYCLONE_NUMBER,YEAR,NAME,NUM_BEST_TRACK_ENTRIES
0,1851-06-25,0000,,HU,28.0,-94.8,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
1,1851-06-25,0600,,HU,28.0,-95.4,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
2,1851-06-25,1200,,HU,28.0,-96.0,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
3,1851-06-25,1800,,HU,28.1,-96.5,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
4,1851-06-25,2100,L,HU,28.2,-96.8,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14


**c) Convert the following fields to integers and display the data types of your dataset to show the changes.**
- MAX_WINDSPEED_KT, MIN_PRESURE_MB, NUM_BEST_TRACK_ENTRIES, NE_34KT, SE_34KT, NW_34_KT, SW_34_KT, NE_50KT, SE_50KT, NW_50_KT, SW_50_KT, NE_64KT, SE_64KT, NW_64_KT, SW_64_KT, RADIUS

In [13]:
conv_str = "MAX_WINDSPEED_KT, MIN_PRESSURE_MB, NUM_BEST_TRACK_ENTRIES, NE_34KT, SE_34KT, NW_34_KT, SW_34_KT, NE_50KT, SE_50KT, NW_50_KT, SW_50_KT, NE_64KT, SE_64KT, NW_64_KT, SW_64_KT, RADIUS"
conv_lst = [e.strip() for e in conv_str.split(',')]

In [14]:
for conv in conv_lst:
    final_df[conv] = pd.to_numeric(final_df[conv])

In [15]:
final_df.dtypes

DATE                       object
TIME_UTC                   object
POINT_TYPE                 object
STATUS                     object
LATITUDE                  float64
LONGITUDE                 float64
MAX_WINDSPEED_KT            int64
MIN_PRESSURE_MB             int64
NE_34KT                     int64
SE_34KT                     int64
NW_34_KT                    int64
SW_34_KT                    int64
NE_50KT                     int64
SE_50KT                     int64
NW_50_KT                    int64
SW_50_KT                    int64
NE_64KT                     int64
SE_64KT                     int64
NW_64_KT                    int64
SW_64_KT                    int64
RADIUS                      int64
BASIN                      object
ATCF_CYCLONE_NUMBER        object
YEAR                       object
NAME                       object
NUM_BEST_TRACK_ENTRIES      int64
dtype: object

**d) Export the dataset as a JSON file where each record is an element of a JSON list.**

In [16]:
json_list = []

for i in range(len(final_df)):
    entry = final_df.iloc[i].to_dict()
    entry_dict = {key: entry[key] for key in entry}
    json_list.append(entry_dict)

In [17]:
with open("hurricane_data.json", 'w') as f:
    json.dump(json_list, f)

In [18]:
with open("hurricane_data.json", 'r') as f:
    json_data = json.load(f)
    
print(json.dumps(json_data[:5], indent=4))

[
    {
        "DATE": "1851-06-25",
        "TIME_UTC": "0000",
        "POINT_TYPE": "",
        "STATUS": "HU",
        "LATITUDE": 28.0,
        "LONGITUDE": -94.8,
        "MAX_WINDSPEED_KT": 80,
        "MIN_PRESSURE_MB": -999,
        "NE_34KT": -999,
        "SE_34KT": -999,
        "NW_34_KT": -999,
        "SW_34_KT": -999,
        "NE_50KT": -999,
        "SE_50KT": -999,
        "NW_50_KT": -999,
        "SW_50_KT": -999,
        "NE_64KT": -999,
        "SE_64KT": -999,
        "NW_64_KT": -999,
        "SW_64_KT": -999,
        "RADIUS": -999,
        "BASIN": "AL",
        "ATCF_CYCLONE_NUMBER": "01",
        "YEAR": "1851",
        "NAME": "UNNAMED",
        "NUM_BEST_TRACK_ENTRIES": 14
    },
    {
        "DATE": "1851-06-25",
        "TIME_UTC": "0600",
        "POINT_TYPE": "",
        "STATUS": "HU",
        "LATITUDE": 28.0,
        "LONGITUDE": -95.4,
        "MAX_WINDSPEED_KT": 80,
        "MIN_PRESSURE_MB": -999,
        "NE_34KT": -999,
        "SE_34KT": -99

**e) Create a table named storms in MongoDB. Choose appropriate fields/values for the partition and sort keys.**

Connect to the DB

In [19]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017")

In [20]:
database = mongo_client['hurricane-data']
collection = database['storms']

Make sure we're working from a blank canvas

In [21]:
collection.drop()
collection = database['storms']
print("Good to go!")

Good to go!


**f) Write your dataset to your DynamoDB storms table with all fields, except the partition key and sort key, stored in a sub-object named INFO.**

In [22]:
i = 0
record_counts = len(final_df)
with open("hurricane_data.json", 'r') as f:
    json_data = json.load(f)

for storm in json_data:
    i += 1
    if not i % 1000:
        print(f"Inserted {i:,} records...")

    id_date_time = f"{storm['ATCF_CYCLONE_NUMBER']} {storm['DATE']} {storm['TIME_UTC']}"

    collection.insert_one(
        {
        "YEAR": storm["YEAR"],
        "ID_DATE_TIME": id_date_time,
        "INFO":{
            "DATE": storm["DATE"],
            "TIME_UTC": storm["TIME_UTC"],
            "POINT_TYPE": storm["POINT_TYPE"],
            "STATUS": storm["STATUS"],
            "LATITUDE": storm["LATITUDE"],
            "LONGITUDE": storm["LONGITUDE"],
            "MAX_WINDSPEED_KT": storm["MAX_WINDSPEED_KT"],
            "MIN_PRESSURE_MB": storm["MIN_PRESSURE_MB"],
            "NE_34KT": storm["NE_34KT"],
            "SE_34KT": storm["SE_34KT"],
            "NW_34_KT": storm["NW_34_KT"],
            "SW_34_KT": storm["SW_34_KT"],
            "NE_50KT": storm["NE_50KT"],
            "SE_50KT": storm["SE_50KT"],
            "NW_50_KT": storm["NW_50_KT"],
            "SW_50_KT": storm["SW_50_KT"],
            "NE_64KT": storm["NE_64KT"],
            "SE_64KT": storm["SE_64KT"],
            "NW_64_KT": storm["NW_64_KT"],
            "SW_64_KT": storm["SW_64_KT"],
            "RADIUS": storm["RADIUS"],
            "BASIN": storm["BASIN"],
            "ATCF_CYCLONE_NUMBER": storm["ATCF_CYCLONE_NUMBER"],
            "NAME": storm["NAME"],
            "NUM_BEST_TRACK_ENTRIES": storm["NUM_BEST_TRACK_ENTRIES"]
            }
        }
    )

print(f"{i:,} records inserted!")

Inserted 1,000 records...
Inserted 2,000 records...
Inserted 3,000 records...
Inserted 4,000 records...
Inserted 5,000 records...
Inserted 6,000 records...
Inserted 7,000 records...
Inserted 8,000 records...
Inserted 9,000 records...
Inserted 10,000 records...
Inserted 11,000 records...
Inserted 12,000 records...
Inserted 13,000 records...
Inserted 14,000 records...
Inserted 15,000 records...
Inserted 16,000 records...
Inserted 17,000 records...
Inserted 18,000 records...
Inserted 19,000 records...
Inserted 20,000 records...
Inserted 21,000 records...
Inserted 22,000 records...
Inserted 23,000 records...
Inserted 24,000 records...
Inserted 25,000 records...
Inserted 26,000 records...
Inserted 27,000 records...
Inserted 28,000 records...
Inserted 29,000 records...
Inserted 30,000 records...
Inserted 31,000 records...
Inserted 32,000 records...
Inserted 33,000 records...
Inserted 34,000 records...
Inserted 35,000 records...
Inserted 36,000 records...
Inserted 37,000 records...
Inserted 3

### Part 2 - Querying and Scanning

**a) Return a total count of records in your DynamoDB storms table.**

In [23]:
items = collection.count_documents({})
items

53501

**b) Given the following legend, which represents the status of a storm, return a cumulative count of records in 2020 and 2021 with the HU classification.**

- TD – Tropical cyclone of tropical depression intensity (< 34 knots)
- TS – Tropical cyclone of tropical storm intensity (34-63 knots)
- HU – Tropical cyclone of hurricane intensity (> 64 knots)
- EX – Extratropical cyclone (of any intensity)
- SD – Subtropical cyclone of subtropical depression intensity (< 34 knots)
- SS – Subtropical cyclone of subtropical storm intensity (> 34 knots)
- LO – A low that is neither a tropical cyclone, a subtropical cyclone, nor an extratropical cyclone (of any intensity)
- WV – Tropical Wave (of any intensity)
- DB – Disturbance (of any intensity)

Get hurricanes from 2020

In [24]:
count = 0
response = collection.find(
    {'$or': [{'YEAR': {'$eq': '2020'}}, {'YEAR': {'$eq': '2021'}}]}
)

hurricanes = list(response)

print(f"There were {len(hurricanes)} storm entries in 2020 and 2021")

There were 1485 storm entries in 2020 and 2021


**3) Given that a Category 4 hurricane has winds ranging from 130-156 mph, return and print out a tabulated list (without duplicates and sorted in ascending order by year) of all Category 4 hurricanes since the year 2000. Print out only the year and the name of the hurricane.**

In [25]:
mph_to_knots = lambda mph: mph / 1.150779448

In [26]:
min_cat_four = mph_to_knots(130)
max_cat_four = mph_to_knots(156)
print(f"Cat 4 {min_cat_four:.2f} - {max_cat_four:.2f} knots")

Cat 4 112.97 - 135.56 knots


Query data over the date range, filter the hurricanes

In [27]:
cat_fours = dict({})

response = collection.find(
    {'$and': 
     [{'YEAR': {'$gte': '2000'}}, 
      {'YEAR': {'$lte': '2022'}}, 
      {'INFO.STATUS': {'$eq': 'HU'}}, 
      {'INFO.MAX_WINDSPEED_KT': {'$gte': min_cat_four}},
      {'INFO.MAX_WINDSPEED_KT': {'$lte': max_cat_four}}]},
    {
        '_id': 0,
        'INFO.NAME': 1,
        'YEAR': 1
    }
)

for item in response:
    year = item['YEAR']
    name = item['INFO']['NAME']
    if name in cat_fours:
        cat_fours[name]['START'] = min(int(year), cat_fours[name]['START'])
        cat_fours[name]['END'] = max(int(year), cat_fours[name]['END'])
    else:
        cat_fours[name] = {'START': int(year), 'END': int(year)}

for four in cat_fours:
    print(f"{four:10}: {cat_fours[four]['START']} - {cat_fours[four]['END']}")

ISAAC     : 2000 - 2000
KEITH     : 2000 - 2000
IRIS      : 2001 - 2001
MICHELLE  : 2001 - 2001
LILI      : 2002 - 2002
FABIAN    : 2003 - 2003
ISABEL    : 2003 - 2003
CHARLEY   : 2004 - 2004
FRANCES   : 2004 - 2004
IVAN      : 2004 - 2004
KARL      : 2004 - 2004
DENNIS    : 2005 - 2005
EMILY     : 2005 - 2005
KATRINA   : 2005 - 2005
RITA      : 2005 - 2005
WILMA     : 2005 - 2005
DEAN      : 2007 - 2007
FELIX     : 2007 - 2007
GUSTAV    : 2008 - 2008
IKE       : 2008 - 2008
OMAR      : 2008 - 2008
PALOMA    : 2008 - 2008
BILL      : 2009 - 2009
DANIELLE  : 2010 - 2010
EARL      : 2010 - 2010
IGOR      : 2010 - 2010
JULIA     : 2010 - 2010
KATIA     : 2011 - 2011
OPHELIA   : 2011 - 2011
GONZALO   : 2014 - 2014
JOAQUIN   : 2015 - 2015
MATTHEW   : 2016 - 2016
NICOLE    : 2016 - 2016
HARVEY    : 2017 - 2017
IRMA      : 2017 - 2017
JOSE      : 2017 - 2017
MARIA     : 2017 - 2017
FLORENCE  : 2018 - 2018
MICHAEL   : 2018 - 2018
DORIAN    : 2019 - 2019
LORENZO   : 2019 - 2019
LAURA     : 2020

Sort and print!

In [28]:
hurrs = [[i, cat_fours[i]['START'], cat_fours[i]['END']] for i in cat_fours]
hurrs.sort(key=lambda x: x[1])

print(f'{"Name":10} | {"Start":6} | {"End":6}')
for h in hurrs:
    print(f'{h[0]:10} | {h[1]:6} | {h[2]:6}')

Name       | Start  | End   
ISAAC      |   2000 |   2000
KEITH      |   2000 |   2000
IRIS       |   2001 |   2001
MICHELLE   |   2001 |   2001
LILI       |   2002 |   2002
FABIAN     |   2003 |   2003
ISABEL     |   2003 |   2003
CHARLEY    |   2004 |   2004
FRANCES    |   2004 |   2004
IVAN       |   2004 |   2004
KARL       |   2004 |   2004
DENNIS     |   2005 |   2005
EMILY      |   2005 |   2005
KATRINA    |   2005 |   2005
RITA       |   2005 |   2005
WILMA      |   2005 |   2005
DEAN       |   2007 |   2007
FELIX      |   2007 |   2007
GUSTAV     |   2008 |   2008
IKE        |   2008 |   2008
OMAR       |   2008 |   2008
PALOMA     |   2008 |   2008
BILL       |   2009 |   2009
DANIELLE   |   2010 |   2010
EARL       |   2010 |   2010
IGOR       |   2010 |   2010
JULIA      |   2010 |   2010
KATIA      |   2011 |   2011
OPHELIA    |   2011 |   2011
GONZALO    |   2014 |   2014
JOAQUIN    |   2015 |   2015
MATTHEW    |   2016 |   2016
NICOLE     |   2016 |   2016
HARVEY     |  

**d) Return the year and maximum sustained winds of the hurricane named Gilbert**

Get knots to mph

In [29]:
knots_to_mph = lambda kt: kt * 1.150779448

Get the data for Gilbert

In [30]:
gilbert_winds = dict({})

response = collection.find(
    {'INFO.NAME': {'$eq': 'GILBERT'}},
    {
        '_id':0,
        'YEAR':1,
        'INFO.MAX_WINDSPEED_KT':1
    }
)

gilbert = list(response)
for i in gilbert:
    year = i['YEAR']
    windspeed = i['INFO']['MAX_WINDSPEED_KT']
    if year in gilbert_winds:
        gilbert_winds[year].append(windspeed)
    else:
        gilbert_winds[year] = [windspeed]

Print results

In [31]:
print("Hurricane Gilbert:")
print(f"Year\n\t- Wind Speed (mph)")
for year in gilbert_winds:
    print(year)
    for speed in gilbert_winds[year]:
        print(f'\t- {speed}')

Hurricane Gilbert:
Year
	- Wind Speed (mph)
1988
	- 25
	- 25
	- 30
	- 30
	- 35
	- 40
	- 45
	- 50
	- 55
	- 65
	- 80
	- 95
	- 100
	- 105
	- 110
	- 110
	- 115
	- 110
	- 110
	- 115
	- 125
	- 140
	- 160
	- 155
	- 145
	- 140
	- 130
	- 100
	- 90
	- 85
	- 90
	- 100
	- 110
	- 115
	- 115
	- 110
	- 80
	- 50
	- 35
	- 30
	- 30
	- 30
	- 25
	- 25
	- 25
	- 25
	- 25
	- 25
	- 25
